In [1]:
## CNN with 4 Convolutional Layers and Batch Normalization

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
#import pandas as pd
np.random.seed(12345)
#%matplotlib inline

Using TensorFlow backend.


In [3]:
train_images_path = keras.utils.get_file('train-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz')
train_labels_path = keras.utils.get_file('train-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-labels-idx1-ubyte.gz')
test_images_path = keras.utils.get_file('t10k-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-images-idx3-ubyte.gz')
test_labels_path = keras.utils.get_file('t10k-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-labels-idx1-ubyte.gz')


In [4]:
batch_size = 512

In [5]:
def load_mnist(images_path, labels_path):
    import os
    import gzip
    import numpy as np

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [6]:
X_train_orig, y_train_orig = load_mnist(train_images_path, train_labels_path)
X_test, y_test = load_mnist(test_images_path, test_labels_path)
X_train_orig = X_train_orig.astype('float32')
X_test = X_test.astype('float32')
X_train_orig /= 255
X_test /= 255

In [7]:
print(X_train_orig.shape)
print(y_train_orig.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, test_size=0.2, random_state=12345)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(48000, 784)
(48000,)
(12000, 784)
(12000,)


In [10]:
#plt.imshow(X_train[1, :].reshape((28, 28)))

In [11]:
img_rows = 28
img_cols = 28
input_shape = (img_rows, img_cols, 1)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

In [12]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [13]:
cnn3 = Sequential([
    Lambda(norm_input, input_shape=(28,28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),

    Conv2D(32, kernel_size=(3, 3), activation='relu'),    
    BatchNormalization(),
    Dropout(0.25),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),    
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    
    Conv2D(128, kernel_size=(3, 3), activation='relu'),    
    BatchNormalization(),
    Dropout(0.25),

    Flatten(),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [14]:
cnn3.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [15]:
cnn3.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 445s 9ms/step - loss: 0.7074 - acc: 0.7573 - val_loss: 0.4109 - val_acc: 0.8479
Epoch 2/10
48000/48000 [==============================] - 313s 7ms/step - loss: 0.4247 - acc: 0.8473 - val_loss: 0.3490 - val_acc: 0.8762
Epoch 3/10
48000/48000 [==============================] - 299s 6ms/step - loss: 0.3502 - acc: 0.8749 - val_loss: 0.2770 - val_acc: 0.9009
Epoch 4/10
48000/48000 [==============================] - 298s 6ms/step - loss: 0.3058 - acc: 0.8906 - val_loss: 0.2314 - val_acc: 0.9189
Epoch 5/10
48000/48000 [==============================] - 298s 6ms/step - loss: 0.2763 - acc: 0.9007 - val_loss: 0.2656 - val_acc: 0.9033
Epoch 6/10
48000/48000 [==============================] - 196s 4ms/step - loss: 0.2545 - acc: 0.9080 - val_loss: 0.2458 - val_acc: 0.9116
Epoch 7/10
48000/48000 [==============================] - 189s 4ms/step - loss: 0.2365 - acc: 0.9147 - val_loss: 0.2037 - 

In [16]:
cnn3.optimizer.lr = 0.0001

In [17]:
cnn3.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 188s 4ms/step - loss: 0.1900 - acc: 0.9308 - val_loss: 0.1931 - val_acc: 0.9318
Epoch 2/10
48000/48000 [==============================] - 200s 4ms/step - loss: 0.1797 - acc: 0.9346 - val_loss: 0.1876 - val_acc: 0.9363
Epoch 3/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.1680 - acc: 0.9391 - val_loss: 0.1984 - val_acc: 0.9325
Epoch 4/10
48000/48000 [==============================] - 328s 7ms/step - loss: 0.1597 - acc: 0.9424 - val_loss: 0.2009 - val_acc: 0.9332
Epoch 5/10
48000/48000 [==============================] - 327s 7ms/step - loss: 0.1519 - acc: 0.9440 - val_loss: 0.1958 - val_acc: 0.9368
Epoch 6/10
48000/48000 [==============================] - 328s 7ms/step - loss: 0.1454 - acc: 0.9477 - val_loss: 0.1925 - val_acc: 0.9357
Epoch 7/10
48000/48000 [==============================] - 329s 7ms/step - loss: 0.1396 - acc: 0.9498 - val_loss: 0.1965 - 

In [18]:
score = cnn3.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.22021763806790112
Test accuracy: 0.9303


In [19]:
# Data Augmentation

In [20]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=batch_size)
val_batches = gen.flow(X_val, y_val, batch_size=batch_size)


In [21]:
cnn3.fit_generator(batches, steps_per_epoch=48000//batch_size, epochs=50,
                    validation_data=val_batches, validation_steps=12000//batch_size, use_multiprocessing=True)


Epoch 1/50
93/93 [==============================] - 339s 4s/step - loss: 0.3993 - acc: 0.8632 - val_loss: 0.2945 - val_acc: 0.8947
Epoch 2/50
93/93 [==============================] - 327s 4s/step - loss: 0.3232 - acc: 0.8840 - val_loss: 0.2788 - val_acc: 0.9009
Epoch 3/50
93/93 [==============================] - 326s 4s/step - loss: 0.3049 - acc: 0.8894 - val_loss: 0.2768 - val_acc: 0.8986
Epoch 4/50
93/93 [==============================] - 328s 4s/step - loss: 0.2945 - acc: 0.8926 - val_loss: 0.2893 - val_acc: 0.8942
Epoch 5/50
93/93 [==============================] - 327s 4s/step - loss: 0.2841 - acc: 0.8989 - val_loss: 0.2510 - val_acc: 0.9108
Epoch 6/50
93/93 [==============================] - 328s 4s/step - loss: 0.2791 - acc: 0.8995 - val_loss: 0.2309 - val_acc: 0.9184
Epoch 7/50
93/93 [==============================] - 328s 4s/step - loss: 0.2667 - acc: 0.9037 - val_loss: 0.2294 - val_acc: 0.9183
Epoch 8/50
93/93 [==============================] - 329s 4s/step - loss: 0.2644 - a

In [22]:
score = cnn3.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.18805894897729158
Test accuracy: 0.9361


In [23]:
cnn3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        18496     
__________